# Notebook — Introduction pratique à **Polars** pour la transformation de données 🧭✨
Un petit notebook concis pour montrer comment lire, transformer, agréger, joindre et sauvegarder des jeux de données NYC taxi avec **Polars** — rapide, mémoire-friendly et prêt pour la production. 🚀

## 0. Prérequis


In [ ]:
!  pip install polars pyarrow requests pandas openpyxl

## 1. Lire les fichiers avec Polars

In [44]:
import polars as pl
trip_path = './../python_project/data/yellow_tripdata/2025-01.parquet'
zones_path = './../python_project/data/yellow_tripdata/taxi_zones.parquet'
# lecture (format parquet + csv)
df = pl.read_parquet(trip_path)
zones = pl.read_parquet(zones_path)

In [26]:
# aperçu et schéma
df.shape

(3475226, 20)

In [27]:
df.schema

Schema([('VendorID', Int32),
        ('tpep_pickup_datetime', Datetime(time_unit='us', time_zone=None)),
        ('tpep_dropoff_datetime', Datetime(time_unit='us', time_zone=None)),
        ('passenger_count', Int64),
        ('trip_distance', Float64),
        ('RatecodeID', Int64),
        ('store_and_fwd_flag', String),
        ('PULocationID', Int32),
        ('DOLocationID', Int32),
        ('payment_type', Int64),
        ('fare_amount', Float64),
        ('extra', Float64),
        ('mta_tax', Float64),
        ('tip_amount', Float64),
        ('tolls_amount', Float64),
        ('improvement_surcharge', Float64),
        ('total_amount', Float64),
        ('congestion_surcharge', Float64),
        ('Airport_fee', Float64),
        ('cbd_congestion_fee', Float64)])

In [8]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee',
 'cbd_congestion_fee']

In [9]:
df.head(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
i32,datetime[μs],datetime[μs],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2025-01-01 00:18:38,2025-01-01 00:26:59,1,1.6,1,"""N""",229,237,1,10.0,3.5,0.5,3.0,0.0,1.0,18.0,2.5,0.0,0.0
1,2025-01-01 00:32:40,2025-01-01 00:35:13,1,0.5,1,"""N""",236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0
1,2025-01-01 00:44:04,2025-01-01 00:46:01,1,0.6,1,"""N""",141,141,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,0.0
2,2025-01-01 00:14:27,2025-01-01 00:20:01,3,0.52,1,"""N""",244,244,2,7.2,1.0,0.5,0.0,0.0,1.0,9.7,0.0,0.0,0.0
2,2025-01-01 00:21:34,2025-01-01 00:25:06,3,0.66,1,"""N""",244,116,2,5.8,1.0,0.5,0.0,0.0,1.0,8.3,0.0,0.0,0.0


## 2. Nettoyage minimal & cast de types (`with_columns`)

In [38]:

df = df.with_columns([
    # si les colonnes datetime sont des strings, str.strptime; sinon cast fonctionne si déjà datetime
    pl.col("tpep_pickup_datetime").alias("pickup_datetime"),
    pl.col("tpep_pickup_datetime").dt.strftime("%Y-%m-%d").alias("pickup_datetime_str"),
    pl.col("tpep_dropoff_datetime").alias("dropoff_datetime"),
    pl.col("passenger_count").cast(pl.Int8),
    pl.col("trip_distance").cast(pl.Float32),
    pl.col("fare_amount").cast(pl.Float32),
    pl.col("tip_amount").cast(pl.Float32),
    pl.col("total_amount").cast(pl.Float32),
])
df

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee,pickup_datetime,pickup_datetime_str,dropoff_datetime
i32,datetime[μs],datetime[μs],i8,f32,i64,str,i32,i32,i64,f32,f64,f64,f32,f64,f64,f32,f64,f64,f64,datetime[μs],str,datetime[μs]
1,2025-01-01 00:18:38,2025-01-01 00:26:59,1,1.6,1,"""N""",229,237,1,10.0,3.5,0.5,3.0,0.0,1.0,18.0,2.5,0.0,0.0,2025-01-01 00:18:38,"""2025-01-01""",2025-01-01 00:26:59
1,2025-01-01 00:32:40,2025-01-01 00:35:13,1,0.5,1,"""N""",236,237,1,5.1,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,0.0,2025-01-01 00:32:40,"""2025-01-01""",2025-01-01 00:35:13
1,2025-01-01 00:44:04,2025-01-01 00:46:01,1,0.6,1,"""N""",141,141,1,5.1,3.5,0.5,2.0,0.0,1.0,12.1,2.5,0.0,0.0,2025-01-01 00:44:04,"""2025-01-01""",2025-01-01 00:46:01
2,2025-01-01 00:14:27,2025-01-01 00:20:01,3,0.52,1,"""N""",244,244,2,7.2,1.0,0.5,0.0,0.0,1.0,9.7,0.0,0.0,0.0,2025-01-01 00:14:27,"""2025-01-01""",2025-01-01 00:20:01
2,2025-01-01 00:21:34,2025-01-01 00:25:06,3,0.66,1,"""N""",244,116,2,5.8,1.0,0.5,0.0,0.0,1.0,8.3,0.0,0.0,0.0,2025-01-01 00:21:34,"""2025-01-01""",2025-01-01 00:25:06
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2,2025-01-31 23:01:48,2025-01-31 23:16:29,null,3.35,null,null,79,237,0,15.85,0.0,0.5,0.0,0.0,1.0,20.6,null,null,0.75,2025-01-31 23:01:48,"""2025-01-31""",2025-01-31 23:16:29
2,2025-01-31 23:50:29,2025-02-01 00:17:27,null,8.73,null,null,161,116,0,28.139999,0.0,0.5,0.0,0.0,1.0,32.889999,null,null,0.75,2025-01-31 23:50:29,"""2025-01-31""",2025-02-01 00:17:27
2,2025-01-31 23:26:59,2025-01-31 23:43:01,null,2.64,null,null,144,246,0,14.91,0.0,0.5,0.0,0.0,1.0,19.66,null,null,0.75,2025-01-31 23:26:59,"""2025-01-31""",2025-01-31 23:43:01


# 3. Filtrage basique

In [29]:
valid = df.filter(
    pl.col("passenger_count") > 0, 
    pl.col("trip_distance") > 0, 
    pl.col("fare_amount") > 0
)

In [30]:
valid.select([
    "pickup_datetime","dropoff_datetime","passenger_count","trip_distance","fare_amount","tip_amount"
]).head(5)

pickup_datetime,dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount
datetime[μs],datetime[μs],i8,f32,f32,f32
2025-01-01 00:18:38,2025-01-01 00:26:59,1,1.6,10.0,3.0
2025-01-01 00:32:40,2025-01-01 00:35:13,1,0.5,5.1,2.02
2025-01-01 00:44:04,2025-01-01 00:46:01,1,0.6,5.1,2.0
2025-01-01 00:14:27,2025-01-01 00:20:01,3,0.52,7.2,0.0
2025-01-01 00:21:34,2025-01-01 00:25:06,3,0.66,5.8,0.0


## 5. Ajouter une colonne calculée (`with_columns`)

In [31]:
valid = valid.with_columns([
    (pl.col("tip_amount") / pl.when(pl.col("fare_amount") == 0).then(1).otherwise(pl.col("fare_amount")))
       .alias("tip_pct")
])
valid.select(["fare_amount","tip_amount","tip_pct"]).head(5)

fare_amount,tip_amount,tip_pct
f32,f32,f32
10.0,3.0,0.3
5.1,2.02,0.396078
5.1,2.0,0.392157
7.2,0.0,0.0
5.8,0.0,0.0


## 6. Agrégations groupées (exemples courants)

In [41]:
# agrégation par PULocationID : nombre de trajets, distance moyenne, montant moyen, tip% moyen
agg = valid.group_by(
        [
            "PULocationID", "pickup_datetime_str"
        ]
        ).agg(
        [
        pl.len().alias("n_trips"),
        pl.sum("fare_amount").alias("total_fare"),
        pl.mean("trip_distance").alias("avg_distance"),
        pl.mean("total_amount").alias("avg_total"),
        pl.mean("tip_pct").alias("avg_tip_pct"),
    ]).sort(
        "n_trips", descending=True
    )  # trier par nombre de trajets décroissant

agg.head(10)

PULocationID,pickup_datetime_str,n_trips,total_fare,avg_distance,avg_total,avg_tip_pct
i32,str,u32,f32,f32,f32,f32
237,"""2025-01-23""",6864,84224.515625,1.600251,20.476576,0.247662
161,"""2025-01-23""",6806,104350.039062,2.106201,25.091021,0.243275
161,"""2025-01-16""",6787,106234.125,2.284286,25.399893,0.246784
161,"""2025-01-30""",6781,107952.898438,2.280676,25.843473,0.242633
161,"""2025-01-15""",6701,101508.890625,2.252846,24.975185,0.247423
161,"""2025-01-22""",6593,96440.945312,2.0152,24.15494,0.248258
237,"""2025-01-30""",6331,80602.867188,1.678368,20.957125,0.239857
161,"""2025-01-14""",6183,92398.007812,2.228246,24.651052,0.243692
132,"""2025-01-20""",6173,391231.84375,16.066435,81.745316,0.149216


## 7. Jointure avec la table des zones (lookup)

In [45]:
# taxi_zone_lookup.csv contient typiquement : LocationID, Borough, Zone, service_zone
# renommer LocationID pour joindre sur PULocationID
zones = zones.rename({"LocationID": "PULocationID"})

# assurez-vous des types (int / même dtype)
zones = zones.with_columns(pl.col("PULocationID").cast(pl.Int32))

agg = agg.join(zones, on="PULocationID", how="left")
agg.select(["PULocationID","Zone","Borough","n_trips","avg_total","avg_tip_pct"]).head(10)

PULocationID,Zone,Borough,n_trips,avg_total,avg_tip_pct
i32,str,str,u32,f32,f32
237,"""Upper East Side South""","""Manhattan""",6864,20.476576,0.247662
161,"""Midtown Center""","""Manhattan""",6806,25.091021,0.243275
161,"""Midtown Center""","""Manhattan""",6787,25.399893,0.246784
161,"""Midtown Center""","""Manhattan""",6781,25.843473,0.242633
161,"""Midtown Center""","""Manhattan""",6701,24.975185,0.247423
161,"""Midtown Center""","""Manhattan""",6593,24.15494,0.248258
237,"""Upper East Side South""","""Manhattan""",6331,20.957125,0.239857
161,"""Midtown Center""","""Manhattan""",6183,24.651052,0.243692
132,"""JFK Airport""","""Queens""",6173,81.745316,0.149216


## 8. Sauvegarder les résultats (parquet / csv / xlsx)

In [24]:
from pathlib import Path
out_dir = Path("./outputs")
out_dir.mkdir(exist_ok=True)

# parquet (recommandé pour volumes importants)
agg.write_parquet(out_dir / "agg_by_pu.parquet")

# csv
agg.write_csv(out_dir / "agg_by_pu.csv")

# xlsx -> conversion via pandas (openpyxl utilisé par pandas)
import pandas as pd
agg.to_pandas().to_excel(out_dir / "agg_by_pu.xlsx", index=False)

In [36]:
import plotly.express as px
fig = px.bar(
    agg,
    x='Zone',
    y='n_trips', 
)
fig.show()